In [14]:
##### add python path #####
import sys
import os
PATH = os.getcwd()
for dir_idx, dir_name in enumerate(PATH.split('/')):
    if 'optimal_control' in dir_name.lower():
        PATH = '/'.join(PATH.split('/')[:(dir_idx+1)])
        break
if not PATH in sys.path:
    sys.path.append(PATH)
###########################

from trajectory_planner import TrajectoryPlanner
from foot_step_planner import FootStepPlanner
from gait_scheduler import GaitScheduler
from env.quadruped.env import Env

from scipy.spatial.transform import Rotation
from scipy.optimize import minimize
from scipy.optimize import Bounds
from copy import deepcopy
import numpy as np
import time

In [2]:
env = Env(enable_draw=False, base_fix=False)
num_leg = env.num_leg
time_step = env.time_step

plan_time_step = 0.05
plan_time_horizon = 20

gs_args = dict()
gs_args['num_leg'] = num_leg
gs_args['gait_period'] = 1.0
gs_args['is_visualize'] = False
gait_scheduler = GaitScheduler(gs_args)

tp_args = dict()
tp_args['time_step'] = plan_time_step
tp_args['time_horizon'] = plan_time_horizon
tp_args['max_accel'] = 0.1
tp_args['max_ang_accel'] = 0.1
trajectory_planner = TrajectoryPlanner(tp_args)

fsp_args = dict()
fsp_args['num_leg'] = num_leg
fsp_args['time_horizon'] = plan_time_horizon
fsp_args['abduct_org_list'] = env.model.abduct_org_list
foot_step_planner = FootStepPlanner(fsp_args)

In [3]:
env.reset()
gait_scheduler.reset()
trajectory_planner.reset()
foot_step_planner.reset()

In [4]:
gait_list = np.zeros((num_leg, 2)) # FR, FL, RR, RL
gait_list[0,0] = 0.5 # offset
gait_list[0,1] = 0.5 # duration
gait_list[2,0] = 0.0 # offset
gait_list[2,1] = 0.5 # duration
gait_scheduler.gait_list = deepcopy(gait_list)
gait_scheduler.update()

vel_cmd = np.zeros(3)
vel_cmd[0] = 0.2
vel_cmd[1] = 0.0
ang_vel_cmd = np.zeros(3)
ang_vel_cmd[2] = 0.0
height_cmd = 0.35

In [5]:
init_state = np.concatenate([env.model.com_pos, env.model.base_rpy, env.model.base_vel, env.model.base_ang_vel])
init_foot_pos_list = deepcopy(env.model.foot_pos_list)

target_trajectory_list = trajectory_planner.get_target_trajectory_list(vel_cmd, ang_vel_cmd, height_cmd, init_state)
contact_list, step_info_list = gait_scheduler.get_contact_list(plan_time_step, plan_time_horizon)
target_foot_step_list = foot_step_planner.get_target_foot_step_list(init_foot_pos_list, step_info_list, target_trajectory_list, \
                                                                    vel_cmd, ang_vel_cmd, height_cmd)

In [35]:
def obj_func(x, args):
    target_trajectory_list = args[0]
    Q_mat = args[1]
    R_mat = args[2]

    time_horizon = int(len(x)/24)
    state_list = x[:int(len(x)/2)].reshape((time_horizon, 12))
    force_list = x[int(len(x)/2):].reshape((time_horizon, 12))
    
    difference_list = state_list - target_trajectory_list
    

    #print(target_trajectory_list.shape)
    f = 0
    return f

def obj_jacobian(x, args):
    jacobian = np.zeros(len(x))
    return jacobian

In [26]:
def eq_cons_func(x):
    e_cons = [0.0]
    return e_cons

eq_cons = {
    'type':'eq',
    'fun' :eq_cons_func,
    }

In [27]:
init_force_list = np.zeros((plan_time_horizon, 12))
init_X = np.concatenate([target_trajectory_list.ravel(), init_force_list.ravel()])

In [28]:
max_force = 1000.0
lowers = np.array([-np.inf]*int(len(init_X)/2) + [-max_force]*int(len(init_X)/2))
uppers = np.array([np.inf]*int(len(init_X)/2) + [max_force]*int(len(init_X)/2))
bounds = Bounds(lowers, uppers)

In [37]:
np.diag([1,2,3])

array([[1, 0, 0],
       [0, 2, 0],
       [0, 0, 3]])

In [38]:
Q_mat = np.eye(12) #np.diag([])
R_mat = np.eye(12) #np.diag([])

In [39]:
res = minimize(obj_func, init_X, args=[target_trajectory_list, Q_mat, R_mat], method="SLSQP", jac=obj_jacobian, bounds=bounds, constraints=[eq_cons], \
               options={'ftol':1e-5, 'disp':True, 'maxiter':10, 'eps':1e-10})

(20, 12) (20, 12)
Singular matrix C in LSQ subproblem    (Exit mode 6)
            Current function value: 0.0
            Iterations: 1
            Function evaluations: 1
            Gradient evaluations: 1
